In [1]:
import pandas as pd


In [2]:
review=pd.read_excel("Review.xlsx")

In [13]:
review.head()

,Review,Label
0,- retail hours (holidays and weekends are gone) -,Negative
1,"Smart people, great company culture, still lot...",Positive
2,"Free food, good culture, good engineers, good ...",Positive
3,"Great work culture , fun place with a lot of p...",Positive
4,Not for the faint of heart - if you are not de...,Negative


In [10]:
review["Review"] = review["Review"].str.replace('#NAME?', '')


In [11]:
import nltk
import random
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords


In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
documents=review.values.astype(str).tolist()

In [12]:
len(review["Review"])

7653

In [43]:
all_words = []



stop_words = list(set(stopwords.words('english')))


allowed_word_types = ["J"]

for p in review["Review"]:
    
    # remove punctuations
    cleaned = re.sub(r'[^(a-zA-Z)\s]','',str(p))
    
    # tokenize 
    tokenized = word_tokenize(cleaned)
    
    # remove stopwords 
    stopped = [w for w in tokenized if not w in stop_words]
    
    # parts of speech tagging for each word 
    pos = nltk.pos_tag(stopped)
    
    # make a list of  all adjectives identified by the allowed word types list above
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())



In [44]:
 len(all_words)

12243

In [52]:
all_words

FreqDist({'lots': 26,
          'transparent': 6,
          'likely': 13,
          'various': 15,
          'right': 47,
          'companys': 5,
          'modern': 4,
          'good': 1013,
          'visible': 9,
          'helpful': 11,
          'impressive': 3,
          'ethical': 2,
          'global': 18,
          'great': 850,
          'active': 5,
          'top': 55,
          'innovative': 45,
          'competitive': 110,
          'personal': 53,
          'new': 176,
          'advisable': 1,
          'easy': 67,
          'info': 2,
          'intern': 15,
          'organisation': 1,
          'stipulated': 1,
          'learn': 13,
          'website': 1,
          'informative': 2,
          'diversified': 6,
          'social': 25,
          'best': 171,
          'aware': 9,
          'responsible': 5,
          'ok': 12,
          'recruiter': 1,
          'ml': 1,
          'clear': 27,
          'open': 56,
          'multicultural': 2,
          'multinat

In [9]:
documents

[['- retail hours (holidays and weekends are gone) -', 'Negative'],
 ['Smart people, great company culture, still lots of room to grow and lots of exciting projects.',
  'Positive'],
 ['Free food, good culture, good engineers, good management', 'Positive'],
 ['Great work culture , fun place with a lot of perks', 'Positive'],
 ['Not for the faint of heart - if you are not dedicated, this job will eat you alive. The standards are high.',
  'Negative'],
 ['Little accountability, too many "owners", lack of team work, continuous re-orgs, too many unnecessary meetings. Back stabbing at the senior level (GM and above).',
  'Negative'],
 ["Managers don't seem to care how hard you work.", 'Negative'],
 ['Love the people, the benefits, the salary - everything! Best job EVER.',
  'Positive'],
 ['Hearing from other fellow interns, I think my own experience was not necessarily the same for everyone else. Some other interns felt their project was going nowhere, or their team was scattered, or that t

In [58]:
all_words = nltk.FreqDist(all_words)

# listing the 10000 most frequent words
word_features = list(all_words.keys())[:10000]

# function to create a dictionary of features for each review in the list document.
# The keys are the words in word_features 
# The values of each key are either true or false for wether that feature appears in the review or not

def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

# Creating features for each review
featuresets = [(find_features(rev), category) for (rev, category) in documents]

# Shuffling the documents 
random.shuffle(featuresets)


In [59]:
 len(featuresets)

5798

In [60]:
training_set = featuresets[:4500]
testing_set = featuresets[4500:]

In [61]:
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI

from statistics import mode
from collections import Counter



In [62]:

classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

classifier.show_most_informative_features(15)

Classifier accuracy percent: 79.58397534668721
Most Informative Features
                    slow = True           Negati : Positi =     48.9 : 1.0
               political = True           Negati : Positi =     38.1 : 1.0
             bureaucracy = True           Negati : Positi =     30.0 : 1.0
                    poor = True           Negati : Positi =     19.2 : 1.0
               stressful = True           Negati : Positi =     18.5 : 1.0
            bureaucratic = True           Negati : Positi =     17.2 : 1.0
                   perks = True           Positi : Negati =     17.2 : 1.0
                     due = True           Negati : Positi =     16.4 : 1.0
                  boring = True           Negati : Positi =     15.8 : 1.0
                    edge = True           Positi : Negati =     15.5 : 1.0
                  little = True           Negati : Positi =     12.9 : 1.0
                    nice = True           Positi : Negati =     12.1 : 1.0
                     too = 

In [63]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)


LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)


MNB_classifier accuracy percent: 80.12326656394453
LogisticRegression_classifier accuracy percent: 79.81510015408321
LinearSVC_classifier accuracy percent: 79.12172573189522
SGDClassifier_classifier accuracy percent: 78.27426810477658
BernoulliNB_classifier accuracy percent: 79.50693374422187


In [69]:
#Highest accuracy is Of MNB_Classifier
save_classifier = open("MNB_classifier1.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

In [70]:
def load_model(file_path): 
    classifier_f = open(file_path, "rb")
    classifier = pickle.load(classifier_f)
    classifier_f.close()
    return classifier

# Multinomial Naive Bayes Classifier 
MNB_Clf2 = load_model('MNB_classifier1.pickle')



In [71]:
feature_list = [f[0] for f in testing_set]

preds = [MNB_Clf2.classify(features) for features in feature_list]

In [74]:
def sentiment(text):
    feats = find_features(text)
    return  MNB_Clf2.classify(feats)

In [79]:
sentiment("yes i can")


'Positive'